In [1]:
import requests
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import json
from bs4 import BeautifulSoup
from pygeocoder import Geocoder

In [2]:
def mcd_scrape(url):
    response = requests.get(url)
    S = BeautifulSoup(response.text, 'lxml')
    mcd = []
    list_items = S.find_all('li')
    for L in list_items:
        if "Panera Bread in" in L.text:
            addr = L.text.replace("  ","")
            addr = addr.replace("\n","")
            addr = addr.split("Panera Bread in")
            mcd.append(addr)
    mcd2 = pd.DataFrame(mcd).drop(0,axis=1).values.tolist()

    mcd3 = []
    for elem in mcd2:
        mcd3.extend(elem)
    mcd4 = []
    for elem in mcd3:
        if elem != None:
            mcd4.append(elem)
    mcd5 = pd.DataFrame(mcd4)
    mcd6 = mcd5[0].str.split(' ',2, expand = True)
    mcd7 = mcd6[2].str.split(' ',1, expand = True).drop(0,axis=1)
    mcd8=mcd6[1].values.tolist()
    mcd9=mcd7.values.tolist()
    mcd9_list = []
    for elem in mcd9:
        mcd9_list.extend(elem)
    mcd10 = pd.DataFrame()
    mcd10['address'] = mcd9_list
    mcd10['zip'] = mcd8
    return(mcd10)

In [3]:
mo_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/panera-bread-4258/us/mo")
mo_mcd['state'] = "MO"
mo_mcd

,address,zip,state
0,1650 Clarkson Road,63017,MO
1,46 Four Seasons Shopping Center,63017,MO
2,238 Chesterfield Mall,63017,MO
3,1650 Clarkson Road,63017,MO
4,46 Four Seasons Shopping Center,63017,MO
...,...,...,...
135,147 Westport Plz,63146,MO
136,290 Mid Rivers Mall Drive,63376,MO
137,290 Mid Rivers Mall Drive,63376,MO
138,290 Mid Rivers Mall Drive,63376,MO


In [4]:
il_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/panera-bread-4258/us/il")
il_mcd['state'] = "IL"
il_mcd

,address,zip,state
0,5 W Rand Rd,60004-3132,IL
1,1734 W Algonquin Rd,60005,IL
2,31 S Evergreen Ave,60005-1427,IL
3,1736 W Algonquin Rd,60005-3405,IL
4,5 W Rand Rd,60004-3132,IL
...,...,...,...
171,3101 W White Oaks Dr,62704,IL
172,700 N Milwaukee Ave,60061,IL
173,447 N Milwaukee Ave,60061,IL
174,700 N Milwaukee Ave,60061,IL


In [5]:
both = [mo_mcd,il_mcd]
both_mcd = pd.concat(both)
cleaned_mcd = pd.DataFrame(both_mcd['zip'].str.split('-').str[0])
cleaned_mcd['address']= both_mcd['address']
cleaned_mcd['state'] = both_mcd['state']
cleaned_mcd = cleaned_mcd[['address', 'state', 'zip']]
cleaned_mcd

,address,state,zip
0,1650 Clarkson Road,MO,63017
1,46 Four Seasons Shopping Center,MO,63017
2,238 Chesterfield Mall,MO,63017
3,1650 Clarkson Road,MO,63017
4,46 Four Seasons Shopping Center,MO,63017
...,...,...,...
171,3101 W White Oaks Dr,IL,62704
172,700 N Milwaukee Ave,IL,60061
173,447 N Milwaukee Ave,IL,60061
174,700 N Milwaukee Ave,IL,60061


In [6]:
cleaned_mcd['location'] = cleaned_mcd[['address', 'state', 'zip']].agg(' '.join, axis=1)
cleaned_mcd

,address,state,zip,location
0,1650 Clarkson Road,MO,63017,1650 Clarkson Road MO 63017
1,46 Four Seasons Shopping Center,MO,63017,46 Four Seasons Shopping Center MO 63017
2,238 Chesterfield Mall,MO,63017,238 Chesterfield Mall MO 63017
3,1650 Clarkson Road,MO,63017,1650 Clarkson Road MO 63017
4,46 Four Seasons Shopping Center,MO,63017,46 Four Seasons Shopping Center MO 63017
...,...,...,...,...
171,3101 W White Oaks Dr,IL,62704,3101 W White Oaks Dr IL 62704
172,700 N Milwaukee Ave,IL,60061,700 N Milwaukee Ave IL 60061
173,447 N Milwaukee Ave,IL,60061,447 N Milwaukee Ave IL 60061
174,700 N Milwaukee Ave,IL,60061,700 N Milwaukee Ave IL 60061


In [7]:
cleaned_mcd['location'].to_csv('Resources/panera_loc.csv', index = False)
check = pd.read_csv('Resources/panera_loc.csv')
check

,location
0,1650 Clarkson Road MO 63017
1,46 Four Seasons Shopping Center MO 63017
2,238 Chesterfield Mall MO 63017
3,1650 Clarkson Road MO 63017
4,46 Four Seasons Shopping Center MO 63017
...,...
311,3101 W White Oaks Dr IL 62704
312,700 N Milwaukee Ave IL 60061
313,447 N Milwaukee Ave IL 60061
314,700 N Milwaukee Ave IL 60061


In [8]:
# used https://dash.geocod.io/ to geocode data and create new csv with lat/long info
geodata = pd.read_csv('Resources/geocoded_panera_loc.csv')
geodata

FileNotFoundError: [Errno 2] File Resources/geocoded_panera_loc.csv does not exist: 'Resources/geocoded_panera_loc.csv'

In [9]:
#insert code here that drops all rows in cleaned_mcd that don't contain the relevant zip codes

zips = []
with open('Resources/zip_list.txt', 'r') as f:
    zips.extend(f.read().split('\n'))
zips

['63001',
 '63005',
 '63006',
 '63011',
 '63017',
 '63021',
 '63022',
 '63024',
 '63025',
 '63026',
 '63031',
 '63032',
 '63033',
 '63034',
 '63038',
 '63040',
 '63042',
 '63043',
 '63044',
 '63045',
 '63074',
 '63088',
 '63099',
 '63105',
 '63114',
 '63117',
 '63119',
 '63121',
 '63122',
 '63123',
 '63124',
 '63125',
 '63126',
 '63127',
 '63128',
 '63129',
 '63130',
 '63131',
 '63132',
 '63133',
 '63134',
 '63135',
 '63136',
 '63137',
 '63138',
 '63140',
 '63141',
 '63143',
 '63144',
 '63145',
 '63146',
 '63151',
 '63167',
 '63198']

In [10]:
complete_mcd = geodata[~geodata.Zip.isin(zips)]
complete_mcd

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
1,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
2,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
3,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
10,33 E Lake St IL 60101,41.931216,-87.987127,1.00,rooftop,33,E Lake St,NaN,NaN,Addison,IL,DuPage County,60101,US,Dupage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1002,6707 W 111th St IL 60482,41.690083,-87.786212,1.00,range_interpolation,6707,W 111th St,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1003,11501 S Harlem Ave IL 60482,41.682778,-87.797175,1.00,range_interpolation,11501,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1004,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau


In [11]:
complete_mcd.to_csv('Resources/final_panera_loc.csv', index = False)
check = pd.read_csv('Resources/final_panera_loc.csv')
check

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
1,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
2,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
3,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
4,33 E Lake St IL 60101,41.931216,-87.987127,1.00,rooftop,33,E Lake St,NaN,NaN,Addison,IL,DuPage County,60101,US,Dupage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
996,6707 W 111th St IL 60482,41.690083,-87.786212,1.00,range_interpolation,6707,W 111th St,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
997,11501 S Harlem Ave IL 60482,41.682778,-87.797175,1.00,range_interpolation,11501,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
998,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
